In [45]:
print("Importing libraries")
import datasets
from datasets import load_dataset, DatasetDict
from huggingface_hub import login
from transformers import WhisperFeatureExtractor
from transformers import WhisperTokenizer
#from transformers import WhisperProcessor
from datasets import Audio

from datasets import Dataset, DatasetDict
import pandas as pd
import gc

#import librosa
import os
import tokenize
import re
import random
from datasets.arrow_writer import ArrowWriter

import numpy as np
from pydub import AudioSegment

whisper_model_string = "whisper-base"
#model_repo_dir = "./whisper-base-sl-artur-fulltest"

train_proportion = 0.90
valid_proportion = 0.10
#test_proportion = 0.05

num_map_threads = 1
samples_per_shard = 500    # number of processed audio samples (log-Mel + labels) per dataset file

cache_base_path = "/mnt/e/datasets/Artur_1_0_full/cache/"
train_cache_path = "/mnt/e/datasets/Artur_1_0_full/cache/train_cache.tmp"
valid_cache_path = "/mnt/e/datasets/Artur_1_0_full/cache/valid_cache.tmp"
test_cache_path = "/mnt/e/datasets/Artur_1_0_full/cache/test_cache.tmp"

dataset_save_path = "/mnt/e/datasets/Artur_1_0_full/processed_dataset/"
train_dataset_save_path = os.path.join(dataset_save_path, "train")
valid_dataset_save_path = os.path.join(dataset_save_path, "valid")
test_dataset_save_path = os.path.join(dataset_save_path, "test")

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/" + whisper_model_string)
tokenizer = WhisperTokenizer.from_pretrained("openai/" + whisper_model_string, language="Slovenian", task="transcribe", predict_timestamps=True)
#processor = WhisperProcessor.from_pretrained("openai/" + whisper_model_string, language="Slovenian", task="transcribe")

Importing libraries


In [46]:
def to_timestamp_token(time):
    time = np.clip(time, a_min=0.0, a_max=30.0)
    time = np.floor(time/0.02) * 0.02   # make timestamp token be in increments of 0.02 second
    token = "<|{:.2f}|>".format(time)
    return token

def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]
    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]
    # encode target text to label ids
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    del batch["audio"]

    return batch

def resample_and_normalize_audio(file_path, target_sr=16000):
    # Load the audio file using pydub
    audio = AudioSegment.from_file(file_path)
    # Resample the audio to the target sampling rate (target_sr)
    audio = audio.set_frame_rate(target_sr)
    # Extract the NumPy array of audio samples
    audio_array = audio.get_array_of_samples()
    # Normalize the audio to the range [-1, 1]
    audio_array = np.array(audio.get_array_of_samples())
    # Normalize the audio to the range [-1, 1]
    audio_array = audio_array.astype('float32') / 32768.0
    return audio_array

def process(audio_samples, test=False):
    # receives as an input a batch of {'audio': {'array': audio_data, 'sampling_rate': sr}, 'sentence': sentence}
    # returns a batch of {'input_features': log_mel_features, 'labels': input_ids}

    """
    num_train = int(len(audio_samples) * train_proportion)
    num_valid = int(len(audio_samples) * valid_proportion)
    num_test = len(audio_samples) - num_train - num_valid
    """
    
    if test==True:
        num_test = len(audio_samples)
    else:
        num_train = int(len(audio_samples) * train_proportion)
        num_valid = len(audio_samples) - num_train
    
    tmp_dataset = DatasetDict()
    """
    tmp_dataset["train"] = Dataset.from_pandas(pd.DataFrame(audio_samples[0:num_train]))
    tmp_dataset["valid"] = Dataset.from_pandas(pd.DataFrame(audio_samples[num_train:num_train+num_valid]))
    tmp_dataset["test"] = Dataset.from_pandas(pd.DataFrame(audio_samples[(num_train+num_valid):]))
    out_dataset = tmp_dataset.map(prepare_dataset, remove_columns=tmp_dataset["train"].column_names, num_proc=num_map_threads, cache_file_names={"train":train_cache_path, "valid":valid_cache_path, "test":test_cache_path})
    """
    if test == True:
        tmp_dataset["test"] = Dataset.from_pandas(pd.DataFrame(audio_samples[0:num_test]))
        out_dataset = tmp_dataset.map(prepare_dataset, remove_columns=tmp_dataset["test"].column_names, num_proc=num_map_threads, cache_file_names={"test":test_cache_path})
    else:
        tmp_dataset["train"] = Dataset.from_pandas(pd.DataFrame(audio_samples[0:num_train]))
        tmp_dataset["valid"] = Dataset.from_pandas(pd.DataFrame(audio_samples[num_train:]))
        out_dataset = tmp_dataset.map(prepare_dataset, remove_columns=tmp_dataset["train"].column_names, num_proc=num_map_threads, cache_file_names={"train":train_cache_path, "valid":valid_cache_path})
    
    [os.remove(os.path.join(cache_base_path, file_name)) for file_name in os.listdir(cache_base_path)]
    return out_dataset

    
def create_audio_sample(audio_samples, sampling_rate, sentence):
    return {
        'audio': {
            'array': audio_samples,
            'sampling_rate': sampling_rate,
        },
        'sentence': sentence,
    }

def find_time_transcript(tr_path):
    with open(tr_path, 'r') as file:
        tr_content = file.read()

    time_transcript_dict = dict({"time1": [], "time2": [], "transcript": [], "len": 0})

    match = re.search(r"startTime=\"0\" endTime=\"([^\"]+)\"", tr_content)
    if match == None:
        return time_transcript_dict
    first_end_time = float(match.group(1))

    found_second_time = True

    while True:
        match1 = re.search("<Sync time=\"([^\"]+)\"/>\n(.+)", tr_content)
        if match1 == None:
            break

        # We make use of "<Background time" tag which is often followed by text
        match1_bg = re.search("<Background time=\"([^\"]+)\".*\n(.+)", tr_content)
        
        if match1_bg != None:
            time1_bg = float(match1_bg.group(1))
            transcript_bg = match1_bg.group(2)
        
        time1 = float(match1.group(1))
        transcript = match1.group(2)

        if match1_bg != None:
            if time1_bg < time1:
                time1 = time1_bg
                match1 = match1_bg
                transcript = transcript_bg

        # Continue normally forward

        if transcript[0] == "<":
            tr_content = tr_content[match1.end(2):]
            continue

        tr_content = tr_content[match1.end(2):]
        if len(tr_content) == 0:
            break
        
        match2 = re.search("<Sync time=\"([^\"]+)\"/>", tr_content)
        if match2 == None:
            if time1 == 0.0:
                time2 = first_end_time
                found_second_time = False
        else:
            time2 = float(match2.group(1))
            #tr_content = tr_content[match2.end(1):]
            #if len(tr_content) == 0:
            #    break
        
        time_transcript_dict["time1"].append(time1)
        time_transcript_dict["time2"].append(time2)
        time_transcript_dict["transcript"].append(transcript)
        time_transcript_dict["len"] += 1

        if found_second_time == False:
            break
    
    return time_transcript_dict

def convert_time_transcript_to_max_30s_segments(trs_path):
    out_dict = dict({"time1": [], "time2": [], "transcript": [], "len": 0 })
    time_transcript_dict = find_time_transcript(trs_path)

    idx = 0
    time1 = 0.0
    transcript_startindex = 0

    while idx < time_transcript_dict["len"]:
        time2 = time_transcript_dict["time2"][idx]
        transcript = ""
        transcript_startindex = idx

        if (time2 - time1) > 30.0:
            # if the segment is longer than 30 seconds, we dispose it
            time1 = time2   # change to new time from which we will look for shorter-than-30s-segments
            idx += 1
            if idx >= time_transcript_dict["len"]:
                break
            time2 = time_transcript_dict["time2"][idx]
        else:
            # segment was shorter than 30 seconds, try add segments while possible
            idx += 1
            transcript_endindex = transcript_startindex
            
            if idx < time_transcript_dict["len"]:
                tmp_time2 = time_transcript_dict["time2"][idx]
                while (tmp_time2 - time1) < 30.0:
                    time2 = tmp_time2
                    transcript_endindex += 1
                    idx +=1
                    if idx >= time_transcript_dict["len"]:
                        break
                    tmp_time2 = time_transcript_dict["time2"][idx]
            
            #transcript = "".join(time_transcript_dict["transcript"][transcript_startindex:transcript_endindex+1])
            
            # add space character if last character in previous part of transcript was a question mark or a dot, etc.
            need_space_chars = "?.,!;:"
            transcript = ""
            for k in range(transcript_startindex, transcript_endindex+1):
                k_tr = time_transcript_dict["transcript"][k]
                if k_tr[-1] in need_space_chars:
                    k_tr += " "
                transcript += k_tr

            out_dict["time1"].append(time1)
            out_dict["time2"].append(time2)
            out_dict["transcript"].append(transcript)
            out_dict["len"] += 1
            
            time1 = time2
        
    return out_dict

#convert_time_transcript_to_max_30s_segments
def get_list_of_transcript_wav_pairs(test=False):
    dataset_path = "/mnt/e/datasets/Artur_1_0_full"
    out_pairs = {"wavpath": [], "trspath": []}
    wav1_base_paths = [os.path.join(dataset_path, "Artur.wav/Artur-B/Artur-B-Brani"),
                       os.path.join(dataset_path, "Artur.wav/Artur-B/Artur-B-Crkovani")]
    
    trs1_base_paths = [os.path.join(dataset_path, "Artur-TRS/Artur-B/Artur-B-Brani"),
                       os.path.join(dataset_path, "Artur-TRS/Artur-B/Artur-B-Crkovani")]

    wav2_base_paths = [os.path.join(dataset_path, "Artur.wav/Artur-J/Artur-J-Splosni-RT"),
                       os.path.join(dataset_path, "Artur.wav/Artur-N/Artur-N-Obrazi-RT"),
                       os.path.join(dataset_path, "Artur.wav/Artur-N/Artur-N-PDom-RT"),
                       os.path.join(dataset_path, "Artur.wav/Artur-N/Artur-N-Prosti-RT"),
                       os.path.join(dataset_path, "Artur.wav/Artur-P/Artur-P-SejeDZ")]
    
    trs2_base_paths = [os.path.join(dataset_path, "Artur-TRS/Artur-J/Artur-J-Splosni-RT"),
                       os.path.join(dataset_path, "Artur-TRS/Artur-N/Artur-N-Obrazi-RT"),
                       os.path.join(dataset_path, "Artur-TRS/Artur-N/Artur-N-PDom-RT"),
                       os.path.join(dataset_path, "Artur-TRS/Artur-N/Artur-N-Prosti-RT"),
                       os.path.join(dataset_path, "Artur-TRS/Artur-P/Artur-P-SejeDZ")]
    
    wav_test_base_path = os.path.join(dataset_path, "Artur.wav/Artur-Test")
    trs_test_base_path = os.path.join(dataset_path, "Artur-TRS/Artur-Test")
    
    if test == True:
        for wav_test_file_name in os.listdir(wav_test_base_path):
            wav_test_file_path = os.path.join(wav_test_base_path, wav_test_file_name)

            split = wav_test_file_name.split("-")
            trs_test_file_name = split[0] + "-" + split[1] + "-" + split[2] + "-" + split[3] + "-std.trs"

            trs_test_file_path = os.path.join(trs_test_base_path, trs_test_file_name)

            out_pairs["wavpath"].append(wav_test_file_path)
            out_pairs["trspath"].append(trs_test_file_path)
        return out_pairs

    for i in range(len(wav1_base_paths)):
        wav1_base_path = wav1_base_paths[i]
        trs1_base_path = trs1_base_paths[i]

        for wav_folder1_name in os.listdir(wav1_base_path):
            wav_folder1_path = os.path.join(wav1_base_path, wav_folder1_name)
            trs_folder1_path = os.path.join(trs1_base_path, wav_folder1_name)

            for wav_file1_name in os.listdir(wav_folder1_path):
                wav_file1_path = os.path.join(wav_folder1_path, wav_file1_name)

                split = wav_file1_name.split("-")
                trs_file1_name = split[0] + "-" + split[1] + "-" + split[2] + "-" + split[3] + "-std.trs"

                trs_file1_path = os.path.join(trs_folder1_path, trs_file1_name)

                out_pairs["wavpath"].append(wav_file1_path)
                out_pairs["trspath"].append(trs_file1_path)

    for i in range(len(wav2_base_paths)):
        wav2_base_path = wav2_base_paths[i]
        trs2_base_path = trs2_base_paths[i]

        for wav2_file_name in os.listdir(wav2_base_path):
            wav2_file_path = os.path.join(wav2_base_path, wav2_file_name)

            split = wav2_file_name.split("-")
            trs2_file_name = split[0] + "-" + split[1] + "-" + split[2] + "-" + split[3] + "-std.trs"

            trs2_file_path = os.path.join(trs2_base_path, trs2_file_name)

            out_pairs["wavpath"].append(wav2_file_path)
            out_pairs["trspath"].append(trs2_file_path)

    return out_pairs

import scipy.io.wavfile

def dataset_generator_v2(test=False):
    audio_samples = []
    sample_rate = 16000
    count = 0

    path_pairs = get_list_of_transcript_wav_pairs(test)
    # we shuffle the paths
    pairs = list(zip(path_pairs["wavpath"], path_pairs["trspath"]))
    random.seed(123)
    random.shuffle(pairs)
    path_pairs["wavpath"], path_pairs["trspath"] = zip(*pairs)
    
    for i in range(len(path_pairs["trspath"])):
        wav_path = path_pairs["wavpath"][i]
        trs_path = path_pairs["trspath"][i]

        # print(trs_path)
        times_transcript_dict = convert_time_transcript_to_max_30s_segments(trs_path)
        y = resample_and_normalize_audio(wav_path)
        max_n = y.shape[0]
        for j in range(times_transcript_dict["len"]):
            t1 = times_transcript_dict["time1"][j]
            t2 = times_transcript_dict["time2"][j]
            n1 = int(t1 * sample_rate)
            n2 = int(t2 * sample_rate)
            # clip values to valid range
            n1 = np.clip(n1, a_min=0, a_max=max_n - 1)
            n2 = np.clip(n2, a_min=0, a_max=max_n - 1)
            
            # temporary bug fix (because of unknown end-of-recording sync time)
            n2_try = max_n
            if (n2_try - n1)/16000.0 < 30.0:
                n2 = n2_try
                t2 = n2/16000.0
            
            derived_y = y[n1:n2]
            transcript = times_transcript_dict["transcript"][j] + to_timestamp_token(t2-t1)

            # debug wav writeout
            #scipy.io.wavfile.write("test{}.wav".format(j), 16000, derived_y)
            #print(j, wav_path, t1, t2, transcript)

            sample = create_audio_sample(derived_y, 16000, transcript)
            audio_samples.append(sample)
            if len(audio_samples) >= samples_per_shard:
                count += 1
                print("count: {} x 500 = {}".format(count, count*500))
                new_dataset = process(audio_samples, test)
                yield new_dataset
                
                del audio_samples
                gc.collect()
                audio_samples = []
        
        #(debug only)
        #if i == 6:
        #    break
    
    if (test and len(audio_samples) >= 1) or (not test and len(audio_samples) >= 2):
        count += 1
        print("count: {} x 500 = {}".format(count, count*500))
        new_dataset = process(audio_samples, test)
        yield new_dataset

# Dataset preparation

In [47]:
gen2 = dataset_generator_v2(test=False)
idx = 1
for batch in gen2:
    batch["train"].to_parquet(os.path.join(train_dataset_save_path, "train_{}.parquet".format(idx)))
    batch["valid"].to_parquet(os.path.join(valid_dataset_save_path, "valid_{}.parquet".format(idx)))
    idx += 1

gen2 = dataset_generator_v2(test=True)
idx = 1
for batch in gen2:
    batch["test"].to_parquet(os.path.join(test_dataset_save_path, "test_{}.parquet".format(idx)))
    idx += 1


count: 1 x 500 = 500


Creating parquet from Arrow format: 100%|██████████| 1/1 [00:01<00:00,  1.23s/ba]


count: 2 x 500 = 1000


Creating parquet from Arrow format: 100%|██████████| 1/1 [00:01<00:00,  1.43s/ba]


count: 3 x 500 = 1500


...

Creating parquet from Arrow format: 100%|██████████| 1/1 [00:14<00:00, 14.47s/ba]


count: 3 x 500 = 1500


Creating parquet from Arrow format: 100%|██████████| 1/1 [00:14<00:00, 14.70s/ba]


count: 4 x 500 = 2000


Creating parquet from Arrow format: 100%|██████████| 1/1 [00:11<00:00, 11.41s/ba]


300000 samples in train/validation dataset  
2000 samples in test dataset

In [48]:
"""
tmp_tokenizer = WhisperTokenizer.from_pretrained("openai/" + whisper_model_string, language="Slovenian", task="transcribe", predict_timestamps=True)
labels = tmp_tokenizer("Dober dan."+to_timestamp_token(32.0)).input_ids
print(labels)
"""

'\ntmp_tokenizer = WhisperTokenizer.from_pretrained("openai/" + whisper_model_string, language="Slovenian", task="transcribe", predict_timestamps=True)\nlabels = tmp_tokenizer("Dober dan."+to_timestamp_token(32.0)).input_ids\nprint(labels)\n'